In [1]:
import yfinance as yf
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
#warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Defining Strategy
## Day Trader Strategy (9-13 minute crossover)
## Swing Trader Strategy (20-100 day crossover)
## Long-term Trader Strategy (50 or 100-200 day crossover)

In [2]:
#def day_trader_strategy(ticker):
    # Get stock data from Yahoo Finance
   

In [3]:
def swing_trader_strategy(MSFT):
    # Get stock data from Yahoo Finance
    stock_data = yf.download(MSFT, period="1y", interval="1d")
    
    # Calculate moving averages
    ema20 = stock_data['Close'].ewm(span=20).mean()
    ema100 = stock_data['Close'].ewm(span=100).mean()
    
    # Create a DataFrame to store signals
    signals = pd.DataFrame(index=stock_data.index)
    signals['position'] = 0
    
    # Determine when to enter and exit the market
    signals['short_ema'] = ema20
    signals['long_ema'] = ema100
    signals['position'][20:] = np.where(signals['short_ema'][20:] > signals['long_ema'][20:], 1, 0)
    signals['position'][100:] = np.where(signals['short_ema'][100:] < signals['long_ema'][100:], -1, signals['position'][100:])
    signals['position'] = signals['position'].ffill().fillna(0)
    
    # Plot the chart
    fig, ax = plt.subplots(figsize=(16,9))
    ax.plot(stock_data.index, stock_data['Close'], label='Close Price')
    # ax.plot(signals.loc[signals.position == 1.0].index, stock_data['Close'][signals.position == 1.0], '^', markersize=10, color='g', label='Buy')
    # ax.plot(signals.loc[signals.position == -1.0].index, stock_data['Close'][signals.position == -1.0], 'v', markersize=10, color='r', label='Sell')
    ax.plot(ema20.index, ema20, label='20 EMA')
    ax.plot(ema100.index, ema100, label='100 EMA')
    ax.legend(loc='upper left')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price ($)')
    plt.show()
    
    return signals


In [4]:
# Download Microsoft's historical price data from Yahoo Finance
msft_df = yf.download('MSFT', start='2020-01-01', end='2023-01-01')
msft_close = msft_df[['Close']]
# Display the DataFrame
print(msft_close)


[*********************100%***********************]  1 of 1 completed
                 Close
Date                  
2020-01-02  160.619995
2020-01-03  158.619995
2020-01-06  159.029999
2020-01-07  157.580002
2020-01-08  160.089996
...                ...
2022-12-23  238.729996
2022-12-27  236.960007
2022-12-28  234.529999
2022-12-29  241.009995
2022-12-30  239.820007

[756 rows x 1 columns]


In [5]:
# Calculate moving averages
ema20 = msft_df['Close'].ewm(span=20).mean()
ema100 = msft_df['Close'].ewm(span=100).mean()
msft_close['ema20'] = ema20 
msft_close ['ema100'] = ema100
display(msft_close)

,Close,ema20,ema100
Date,,,
2020-01-02,160.619995,160.619995,160.619995
2020-01-03,158.619995,159.569995,159.609995
2020-01-06,159.029999,159.371712,159.412784
2020-01-07,157.580002,158.854473,158.940752
2020-01-08,160.089996,159.153335,159.179885
...,...,...,...
2022-12-23,238.729996,244.751423,248.952866
2022-12-27,236.960007,244.009383,248.715383
2022-12-28,234.529999,243.106585,248.434485


In [ ]:
def EMA_crossover_strategy(msft_close, short_window, long_window):
    signals = pd.DataFrame(index=msft_close.index)
    signals['price'] = msft_close['Close']
    signals['short_EMA'] = ema(msft_close['Close'], short_window)
    signals['long_EMA'] = SMA(msft_close['Close'], long_window)
    signals['signal'] = 0.0

    signals['signal'][short_window:] = np.where(signals['short_SMA'][short_window:] > signals['long_SMA'][short_window:], 1.0, 0.0)

    signals['positions'] = signals['signal'].diff()

    return signals

In [9]:
#def SwingTrader(data, window):
    #return data.rolling(window=window).mean()

def EMA_crossover_strategy(msft_close, short_window, long_window):
    signals = pd.DataFrame(index=msft_close.index)
    signals['price'] = msft_close['Close']
    signals['short_EMA'] = ema(msft_close['Close'], short_window)
    signals['long_EMA'] = SMA(msft_close['Close'], long_window)
    signals['signal'] = 0.0

    signals['signal'][short_window:] = np.where(signals['short_SMA'][short_window:] > signals['long_SMA'][short_window:], 1.0, 0.0)

    signals['positions'] = signals['signal'].diff()

    return signals

def plot_signals(data, signals):
    fig, ax = plt.subplots()

    ax.plot(data.index, msft_close['Close'], label='Price')
    ax.plot(data.index, signals['short_SMA'], label='Short SMA')
    ax.plot(data.index, signals['long_SMA'], label='Long SMA')

    ax.plot(signals.loc[signals.positions == 1.0].index, signals.short_SMA[signals.positions == 1.0], '^', markersize=10, color='g', label='Buy')
    ax.plot(signals.loc[signals.positions == -1.0].index, signals.short_SMA[signals.positions == -1.0], 'v', markersize=10, color='r', label='Sell')

    ax.legend(loc='best')
    plt.show()

# Load historical price data (replace 'your_data.csv' with your own data file)
#data = pd.read_csv('your_data.csv', index_col='Date', parse_dates=True)

# Test the SMA crossover strategy with desired window lengths
short_window = 50
long_window = 200
signals = SMA_crossover_strategy(msft_close, short_window, long_window)

# Plot the signals on a chart
plot_signals(msft_close, signals)


NameError: name 'SMA' is not defined

In [ ]:
  # Create a DataFrame to store signals
    signals = pd.DataFrame(index=stock_data.index)
    signals['position'] = 0
    
    # Determine when to enter and exit the market
    signals['short_ema'] = ema20
    signals['long_ema'] = ema100
    signals['position'][20:] = np.where(signals['short_ema'][20:] > signals['long_ema'][20:], 1, 0)
    signals['position'][100:] = np.where(signals['short_ema'][100:] < signals['long_ema'][100:], -1, signals['position'][100:])
    signals['position'] = signals['position'].ffill().fillna(0)
    

In [ ]:
#def long_term_trader_strategy(ticker):
    # Get stock data from Yahoo Finance
   

# Command Line Interface
## 1. Choose Strategy (Day Trader, Swing Trader, Long-term Trader)
## 2. Choose Ticker
## 3. To-do: Choose dates 

In [ ]:
# Ask user for input
#ticker = input("Enter the ticker symbol of the stock: ")
#strategy_choice = input("Choose a strategy (1 = day trader, 2 = swing trader, 3 = long-term trader): ")

# Backtesting function
def backtest(signals, stock_data, initial_capital=10000):
    positions = pd.DataFrame(index=signals.index).fillna(0)
    positions['stock'] = signals['position'] * stock_data['Close']
    portfolio = positions.multiply(stock_data['Close'], axis=0)
    pos_diff = positions.diff()
    portfolio['holdings'] = (positions.multiply(stock_data['Close'], axis=0)).sum(axis=1)
    portfolio['cash'] = initial_capital - (pos_diff.multiply(stock_data['Close'], axis=0)).sum(axis=1).cumsum()
    portfolio['total'] = portfolio['cash'] + portfolio['holdings']
    portfolio['returns'] = portfolio['total'].pct_change()
    return portfolio

# Execute the chosen EMA crossover strategy and plot the results
if strategy_choice == "1":
    signals = day_trader_strategy(ticker)
    stock_data = yf.download(ticker, period="1d", interval="5m")
elif strategy_choice == "2":
    signals = swing_trader_strategy(ticker)
    stock_data = yf.download(ticker, period="1y", interval="1d")
elif strategy_choice == "3":
    signals = long_term_trader_strategy(ticker)
    stock_data = yf.download(ticker, period="5y", interval="1d")
else:
    print("Invalid strategy choice.")
    stock_data = None
    
# Perform backtesting and print the results
if stock_data is not None:
    print(signals)
    portfolio = backtest(signals, stock_data)
    print("\nBacktesting Results:")
    print(portfolio)
    
# Re-enable SettingWithCopyWarning
pd.options.mode.chained_assignment = 'warn'

# II. EXECUTING STRATEGY
## Identify when the long-term and short-term moving averages cross and define as a buy or sell opportunity

Inputs: Moving average dataframe
Outputs: Prices at crossovers and identifying buy versus sell, plot of buy/sell signals


In [ ]:
# Entry Price for buying is when 5 minutes prices or closing prices cross over from 0 to 1
# Exit price for selling is when it goes from 0 or 1 to -1

In [ ]:
# signals_test_df = signals

You can use the signals_test_df to test your function

In [ ]:
 # Function to create a dataframe 

# def buy_sell_opportunity(moving_avg_df):
    
#     moving_avg_df['Buy_Sell'] = np.where(moving_avg_df['position'] == 1, 'Buy', 
#                                            np.where(moving_avg_df['position'] == -1, 'Sell or Out of Position', 'No Position'))
    
#     # Create columns to output for user
#     # moving_avg_df['entry_price'] = 
#     # moving_avg_df['Exit_Price'] = 
    
#     moving_avg_df = moving_avg_df[['Buy_Sell']]
    
#     return moving_avg_df

In [ ]:
# # Calling the function

# buy_sell_opportunity(moving_avg_df = signals)

In [ ]:
# # Call the function you just created! Using the signals dataset that was created abov
# stock_data_test = stock_data
# stock_data_test['profit_loss'] = (stock_data_test['Close'] - stock_data_test['Open']) * stock_data_test['Volume']
# stock_data_test['profit_loss_ind'] = np.where(stock_data_test['profit_loss'] > 0, 'Profit', 'Loss')
# stock_data_test

# III. CALCULATING PROFITABILITY
## Calculating the profitability of the strategy within the given time frame
Thoughts: price comparisons begin on date of first trigger


Inputs: Moving average dataframe
Outputs: Prices at crossovers and identifying buy versus sell, plot of buy/sell signals

In [ ]:
# stock_data

In [ ]:
 signals

In [ ]:
signals.drop(columns=['short_ema','long_ema',], inplace=True, axis=1)

In [ ]:
signals_next = signals.shift(-1)

In [ ]:
signals_next

In [ ]:
signals_next.rename(columns={'position':'position_next'}, inplace=True)

In [ ]:
signals_next.rename(columns={'closed':'closed_next'}, inplace=True)

In [ ]:
signals_merged = pd.merge(left=signals,right=signals_next,left_on='Datetime',right_on='Datetime')

In [ ]:
signals_merged.tail(50)

In [ ]:
filtered_signals_merged = signals_merged[signals_merged['position'] != signals_merged['position_next']]

In [ ]:
filtered_signals_merged.reset_index(inplace=True)

In [ ]:
filtered_signals_merged

In [ ]:
filtered_signals_merged['row_num']= np.arange(len(filtered_signals_merged))+1

In [ ]:
filtered_signals_merged

In [ ]:
pnl = pd.merge(left=filtered_signals_merged,right=filtered_signals_merged,right_index=True,left_on="row_num")

In [ ]:
pnl

In [ ]:
pnl = pnl[pnl.row_num%2==1]

In [ ]:
pnl.drop(columns=['row_num','position_x','closed_x','position_next_x','row_num_x','position_y','closed_y','position_next_y','row_num_y'], inplace=True, axis=1)

In [ ]:
pnl

In [ ]:
pnl.rename(columns={'Datetime_x':'entry_time'}, inplace=True)

In [ ]:
pnl.rename(columns={'closed_next_x':'entry_price'}, inplace=True)

In [ ]:
pnl.rename(columns={'Datetime_y':'exit_time'}, inplace=True)

In [ ]:
pnl.rename(columns={'closed_next_y':'exit_price'}, inplace=True)

In [ ]:
stats = pnl

In [ ]:
stats

In [ ]:
# Function to create a dataframe 

# def buy_sell_opportunity(moving_avg_df):
    
    
    
#     return moving_avg_df